# Agregación y agrupación

Una pieza esencial en el análisis de grandes datos es un resumen eficiente: calcular agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()``, y ``max()``, en las que un único número da una idea de la naturaleza de un conjunto de datos potencialmente grande.
En esta sección, exploraremos las agregaciones en Pandas, desde operaciones simples similares a las que hemos visto en los arrays de NumPy, hasta operaciones más sofisticadas basadas en el concepto de ``groupby``.

Por comodidad, utilizaremos la misma función mágica ``display`` que hemos visto en secciones anteriores:

In [ ]:
import numpy as np
import pandas as pd

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Datos de Planetas

Aquí usaremos el conjunto de datos Planetas, disponible a través del [paquete Seaborn](http://seaborn.pydata.org/) (ver [Visualization With Seaborn](14_Visualization-With-Seaborn.ipynb)).
Proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como *planetas extrasolares* o *exoplanetas* para abreviar). Puede descargarse con un simple comando Seaborn:

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
planets.info()

In [ ]:
planets.head()

Contiene algunos detalles sobre los más de 1.000 planetas extrasolares descubiertos hasta 2014.

## Agregación simple en Pandas

Anteriormente, hemos explorado algunas de las agregaciones de datos disponibles para arrays NumPy (["Agregaciones: Mín, Máx, y todo lo que hay en medio"](4_Computation-on-arrays-aggregates.ipynb)).
Al igual que con un array unidimensional NumPy, para una ``Serie`` Pandas los agregados devuelven un único valor:

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

In [ ]:
ser.sum()

In [ ]:
ser.mean()

Para un ``DataFrame``, por defecto los agregados devuelven resultados dentro de cada columna:

In [ ]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

In [ ]:
df.mean()

Especificando el argumento ``axis``, puede agregar dentro de cada fila:

In [ ]:
df.mean(axis='columns')

Las ``Series`` y ``DataFrame`` de Pandas incluyen todos los agregados comunes mencionados en [Aggregations: Min, Max, and Everything In Between](4_Computation-on-arrays-aggregates.ipynb); además, hay un método de conveniencia ``describe()`` que calcula varios agregados comunes para cada columna y devuelve el resultado.
Usemos esto en los datos de Planetas, por ahora eliminando las filas con valores perdidos:

In [ ]:
planets.dropna().describe()

Esta puede ser una forma útil de empezar a comprender las propiedades generales de un conjunto de datos.
Por ejemplo, en la columna ``year`` vemos que, aunque ya en 1989 se descubrieron exoplanetas, la mitad de los conocidos no se descubrieron hasta 2010 o después.
Esto se debe en gran parte a la misión *Kepler*, un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

La siguiente tabla resume algunas otras agregaciones incorporadas en Pandas:

| Agregacion               | Descripcion                     |
|--------------------------|---------------------------------|
| ``count()``              | Número total de artículos       |
| ``first()``, ``last()``  | Primer y último punto           |
| ``mean()``, ``median()`` | Media y mediana                 |
| ``min()``, ``max()``     | Mínimo y máximo                 |
| ``std()``, ``var()``     | Desviación estandar y varianza  |
| ``mad()``                | Desviación media absoluta       |
| ``prod()``               | Producto de todos los artículos |
| ``sum()``                | Suma de todas las partidas      |

Todos estos son métodos de los objetos ``DataFrame`` y ``Series``.

Sin embargo, para profundizar en los datos, los agregados simples no suelen ser suficientes.
El siguiente nivel de integración de datos es la operación ``groupby``, que permite calcular rápida y eficazmente agregados sobre subconjuntos de datos.

## GroupBy: Dividir, Aplicar, Combinar

Las agregaciones simples pueden darle una idea de su conjunto de datos, pero a menudo preferiríamos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en la llamada operación ``groupby``.
El nombre "group by" procede de un comando del lenguaje de bases de datos SQL, pero quizá sea más ilustrativo pensar en él en los términos acuñados por Hadley Wickham, famoso por Rstats: *dividir, aplicar, combinar*.

### Split, apply, combine

Un ejemplo canónico de esta operación dividir-aplicar-combinar, donde el "aplicar" es una agregación sumatoria.

Esto aclara lo que consigue el ``groupby``:

- El paso *split* consiste en dividir y agrupar un ``DataFrame`` en función del valor de la clave especificada.
- El paso *apply* implica calcular alguna función, normalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso *combinar* combina los resultados de estas operaciones en una matriz de salida.

Aunque esto podría hacerse manualmente utilizando alguna combinación de los comandos de enmascaramiento, agregación y fusión descritos anteriormente, es importante tener en cuenta que *las divisiones intermedias no necesitan instanciarse explícitamente*. En su lugar, ``GroupBy`` puede (a menudo) hacer esto en una sola pasada sobre los datos, actualizando la suma, media, recuento, min, u otro agregado para cada grupo a lo largo del camino.
El poder del ``GroupBy`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo bajo el capó, sino que piensa en la *operación en su conjunto*.

Como ejemplo concreto, echemos un vistazo al uso de Pandas para el cálculo mostrado en este diagrama.
Empezaremos creando el ``DataFrame`` de entrada:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'VV': range(6)})
df

La operación más básica de dividir-aplicar-combinar puede calcularse con el método ``groupby()`` de ``DataFrame``s, pasando el nombre de la columna clave deseada:

In [ ]:
df.groupby('department')

Observa que lo que se devuelve no es un conjunto de ``DataFrame``s, sino un objeto ``DataFrameGroupBy``.
Este objeto es donde está la magia: se puede pensar en él como una vista especial del ``DataFrame``, que está preparado para profundizar en los grupos, pero no hace ningún cálculo real hasta que se aplica la agregación.
Este enfoque de "evaluación perezosa" significa que los agregados comunes pueden implementarse muy eficientemente de un modo casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicar/combinar para producir el resultado deseado:

In [ ]:
df_grouped = df.groupby('department', as_index=False).max()
df_grouped

El método ``sum()`` es sólo una posibilidad aquí; puedes aplicar virtualmente cualquier función de agregación común de Pandas o NumPy, así como virtualmente cualquier operación válida de ``DataFrame``, como veremos en la siguiente discusión.

### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible.
En muchos sentidos, puedes simplemente tratarlo como si fuera una colección de ``DataFrame``s, y hace las cosas difíciles bajo el capó. Veamos algunos ejemplos utilizando los datos de Los Planetas.

Quizás las operaciones más importantes disponibles en un ``GroupBy`` son *agregar*, *filtrar*, *transformar* y *aplicar*.

#### Indexación de columnas

El objeto ``GroupBy`` soporta la indexación por columnas de la misma forma que el ``DataFrame``, y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo

In [ ]:
planets.groupby('method')

In [ ]:
planets.groupby('method')['orbital_period']

Aquí hemos seleccionado un grupo ``Series`` particular del grupo original ``DataFrame`` por referencia a su nombre de columna.

Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [ ]:
planets.head()

In [ ]:
len(planets['method'].unique())

In [ ]:
planets.groupby('method', as_index=False)[['orbital_period']].mean()

In [ ]:
# planets[planets['method']=='Astrometry'][['orbital_period']].mean()

Esto da una idea de la escala general de periodos orbitales (en días) a la que es sensible cada método.

#### Iteración sobre grupos

El objeto ``GroupBy`` permite la iteración directa sobre los grupos, devolviendo cada grupo como una ``Series`` o ``DataFrame``:

In [ ]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Esto puede ser útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido utilizar la funcionalidad incorporada ``apply``, que discutiremos momentáneamente.

#### Métodos de envío

A través de la magia de las clases de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será pasado y llamado en los grupos, ya sean objetos ``DataFrame`` o ``Series``.
Por ejemplo, puedes utilizar el método ``describe()`` de ``DataFrame`` para realizar un conjunto de agregaciones que describan cada grupo en los datos:

In [ ]:
planets.groupby('method')['year'].describe().unstack()

Observar esta tabla nos ayuda a entender mejor los datos: por ejemplo, la gran mayoría de los planetas se han descubierto por los métodos de Velocidad Radial y Tránsito, aunque este último no se hizo común (debido a nuevos telescopios más precisos) hasta la última década.
Los métodos más recientes parecen ser el de la Variación Temporal del Tránsito y el de la Modulación del Brillo Orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Este es solo un ejemplo de la utilidad de los métodos de envío.
Fíjate en que se aplican *a cada grupo individual*, y los resultados se combinan dentro de ``GroupBy`` y se devuelven.
De nuevo, cualquier método válido de ``DataFrame`` o ``Series`` puede utilizarse en el objeto ``GroupBy`` correspondiente, lo que permite realizar operaciones muy flexibles y potentes.

### Agregar, filtrar, transformar, aplicar

La discusión anterior se centró en la agregación para la operación combinar, pero hay más opciones disponibles.
En particular, los objetos ``GroupBy`` tienen métodos ``aggregate()``, ``filter()``, ``transform()``, y ``apply()`` que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

Para el propósito de las siguientes subsecciones, utilizaremos este ``DataFrame``:

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

#### Agregación

Ahora estamos familiarizados con las agregaciones ``GroupBy`` con ``sum()``, ``median()``, y similares, pero el método ``aggregate()`` permite aún más flexibilidad.
Puede tomar una cadena, una función o una lista de ellas y calcular todos los agregados a la vez.
He aquí un ejemplo rápido que combina todos estos métodos:

In [ ]:
df.groupby('key').aggregate(['min', 'median', 'max'])

Otro patrón útil es pasar un diccionario que asigne los nombres de las columnas a las operaciones que deben aplicarse a esa columna:

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

#### Filtrado

Una operación de filtrado permite descartar datos en función de las propiedades del grupo.
Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar es mayor que algún valor crítico:

In [ ]:
def filter_func(x):
    return x['data2'].min() > 1

In [ ]:
df

In [ ]:
filter_func(df)

In [ ]:
display('df', "df.groupby('key').min()", "df.groupby('key').filter(filter_func)")

In [ ]:
df[df['data2'] > 1]

In [ ]:
df

La función de filtrado debe devolver un valor booleano que especifique si el grupo pasa el filtrado. En este caso, como el grupo A no tiene una desviación típica superior a 4, se elimina del resultado.

#### Transformación

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinar.
Para tal transformación, la salida tiene la misma forma que la entrada.
Un ejemplo común es centrar los datos restando la media del grupo:

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

In [ ]:
df.groupby('key').mean()

In [ ]:
df.mean()

In [ ]:
df['data1_transformed'] = df['data1'] - df['data1'].mean()
df

#### El método apply()

El método ``apply()`` permite aplicar una función arbitraria a los resultados del grupo.
La función debe tomar un ``DataFrame``, y devolver un objeto Pandas (por ejemplo, ``DataFrame``, ``Series``) o un escalar; la operación de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una ``apply()`` que normaliza la primera columna por la suma de la segunda:

In [ ]:
def norm_by_data2(x):
    # x es un DataFrame de valores de grupo
    # x['data1'] = x['data1'] / x['data2'].sum()
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

In [ ]:
df.groupby('key').sum()

In [ ]:
print(3/8)

La función ``apply()`` dentro de un ``GroupBy`` es bastante flexible: el único criterio es que la función toma un ``DataFrame`` y devuelve un objeto Pandas o un escalar; ¡lo que hagas en medio depende de ti!

### Especificar la clave de división

En los ejemplos sencillos presentados anteriormente, dividimos el ``DataFrame`` en un único nombre de columna.
Esta es sólo una de las muchas opciones por las que los grupos se pueden definir, y vamos a ir a través de algunas otras opciones para la especificación de grupo aquí.

#### Una lista, matriz, serie o índice que proporcione las claves de agrupación.

La clave puede ser cualquier serie o lista cuya longitud coincida con la del ``DataFrame``. Por ejemplo:

In [ ]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

Por supuesto, esto significa que hay otra forma más verbosa de realizar el ``df.groupby('clave')`` de antes:

In [ ]:
display('df', "df.groupby(df['key']).sum()")

#### Un diccionario o serie que asigna un índice a un grupo

Otro método consiste en proporcionar un diccionario que asigne los valores del índice a las claves del grupo:

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

#### Cualquier función Python

De forma similar al mapeo, puede pasar cualquier función de Python que introduzca el valor del índice y genere el grupo:

In [ ]:
display('df2', 'df2.groupby(str.lower).mean()')

#### Una lista de claves válidas

Además, cualquiera de las opciones de clave anteriores puede combinarse para agruparse en un multiíndice:

In [ ]:
df2.groupby([str.lower, mapping]).mean()

### Ejemplo de agrupación

Como ejemplo de esto, en un par de líneas de código Python podemos juntar todo esto y contar los planetas descubiertos por método y por década:

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

Esto demuestra el poder de la combinación de muchas de las operaciones que hemos discutido hasta ahora cuando se observan conjuntos de datos realistas.
Inmediatamente obtenemos una comprensión general de cuándo y cómo se han descubierto planetas en las últimas décadas.

Aquí sugeriría profundizar en estas pocas líneas de código, y evaluar los pasos individuales para asegurarse de que entiende exactamente lo que están haciendo al resultado.
Ciertamente es un ejemplo algo complicado, pero entender estas piezas te dará los medios para explorar de forma similar tus propios datos.